In [1]:
%pip install seaborn
%pip install sklearn.dataset
%pip install scikit-learn

     -------------------------------------- 293.3/293.3 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 7.6/7.6 MB 3.3 MB/s eta 0:00:00
     -------------------------------------- 163.0/163.0 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 318.0 kB/s eta 0:00:00
     -------------------------------------- 55.4/55.4 kB 960.7 kB/s eta 0:00:00
     ---------------------------------------- 98.3/98.3 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import nltk 
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import pickle

In [3]:
df_data = pd.read_csv("C:/Users/abcd/Downloads/FINAL_PM_Modi_Speech_Cleaned_string.csv")

In [4]:
df_data

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,['governor andhra pradesh shri e l narasimhan ...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,['chief minister gujarat shri vijay rupani ji ...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,['delighted gift city inaugurate india first i...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,['congratulate finance minister arun jaitley j...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,['sri pejavar math respected sri vishwesh tirt...,sri pejavar math respected sri vishwesh tirth ...
...,...,...,...,...,...,...
1177,NMS1178,2023-04-09,My colleagues in the Union Cabinet Shri Bhupen...,English rendering of PM’s address at the inaug...,['colleague union cabinet shri bhupender yadav...,colleague union cabinet shri bhupender yadav j...
1178,NMS1179,2023-04-12,Namaskar! \n\nThe campaign to provide governme...,English rendering of PM’s address at training ...,"['namaskar', 'campaign provide government job ...",namaskar campaign provide government job youth...
1179,NMS1180,2023-04-12,"Namaskar, Governor of Rajasthan Shri Kalraj Mi...",English rendering of PM’s address during flagg...,['namaskar governor rajasthan shri kalraj mish...,namaskar governor rajasthan shri kalraj mishra...
1180,NMS1181,2023-04-14,"Governor of Assam Shri Gulab Chand Kataria ji,...",English rendering of PM’s address at inaugurat...,['governor assam shri gulab chand kataria ji c...,governor assam shri gulab chand kataria ji chi...


In [5]:
#defining a function that would train model
def model(corpus):
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]
    model = Word2Vec(sentences=corpus_tokenized, vector_size=300, window=5, min_count=5, workers=4)
    return model 


In [6]:
# Defining the function for vectorization 
def predict(model, speech):
    vectors = []

    for sentence in speech:
        sent_vec = []
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                sent_vec.append(word_vector)
        vectors.append(sent_vec)

    return vectors 


In [7]:
# Defining a function to count the number of unique words in each sentence
def count_unique_words(j):
    m = j.replace('[','').replace(']','').replace("'","").split(', ')
    l = []
    for k in m:
        l += k.split(" ")
    length = len(l)
    return length 


In [8]:
# Creating the corpus of all the speeches to train the model 
corpus_Modi = []
for i in df_data['Cleaned_Speech']:
    corpus_Modi += i.replace('[','').replace(']','').replace("'","").split(', ')


In [9]:
empty_count = 0
for element in corpus_Modi:
    if element.strip() == '':
        empty_count += 1

print(f'Total empty elements in the list: {empty_count}')

Total empty elements in the list: 579


In [10]:
len(corpus_Modi)

249435

In [11]:
corpus_Modi = [element for element in corpus_Modi if element.strip() != '']

In [12]:
len(corpus_Modi)

248856

In [76]:
# Training the model 
model_modi = model(corpus_Modi)

In [78]:
#creating a model pickle
Word2vec_model_NM = open("Word2vec_model_NM.pickle", "wb")
pickle.dump(model_modi, Word2vec_model_NM)
Word2vec_model_NM.close()

In [14]:
# Creating the list containing each of the speeches
speech_modi = []
for i in df_data['Cleaned_Speech']:
    speech_modi.append(i.replace('[','').replace(']','').replace("'","").split(', '))


In [15]:
len(speech_modi)

1182

In [20]:
count = 0
for sp in speech_modi:
    for sent in sp:
        if len(sent)==0:
            count +=1


In [21]:
count

0

In [18]:
speech_modi = [[sent for sent in sp if len(sent) > 0] for sp in speech_modi]

In [19]:
len(speech_modi)

1182

In [22]:
# Vectorize each speech
speech_modi_vector = []
for sp in speech_modi:
    speech_modi_vector.append(predict(model_modi, sp)) 

In [23]:
import pandas as pd
import numpy as np

# initialize an empty list to store the row dictionaries
rows = []

# loop over all speeches, sentences, and words
for speech_number, speech in enumerate(speech_modi_vector):
    for sentence_number, sentence in enumerate(speech):
        # initialize a vector of zeros with 300 dimensions
        sentence_vector = np.zeros(300)
        for word in sentence:
            # add the ith element of the word vector to the corresponding dimension of the sentence vector
            sentence_vector += np.array(word)
        # divide the sentence vector by the number of words to get the average
        sentence_vector /= len(sentence)
        # create a dictionary with the speech number, sentence number, and sentence vector
        row_dict = {"speech_number": speech_number, "sentence_number": sentence_number}
        for i, dim in enumerate(sentence_vector):
            row_dict["dim_" + str(i)] = dim
        # add the dictionary to the list of rows
        rows.append(row_dict)

# create a dataframe from the list of rows
df = pd.DataFrame(rows)


In [24]:
df

,speech_number,sentence_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,0,0,-0.198454,-0.000900,-0.116400,0.224131,-0.328203,0.123283,0.112045,-0.014164,...,0.131730,0.578573,-0.260603,-0.339803,0.373078,0.258961,0.105751,0.034498,0.041146,0.211072
1,0,1,-0.156093,-0.035716,-0.096698,0.124847,-0.264158,0.093162,0.096666,-0.026142,...,0.115278,0.517290,-0.212438,-0.257892,0.325454,0.294580,0.131161,0.021719,0.021699,0.187387
2,0,2,-0.160581,-0.001085,-0.050404,0.173150,-0.250315,0.073999,0.164324,0.025515,...,0.126147,0.494890,-0.169395,-0.258634,0.311033,0.243183,0.126038,-0.016573,0.047996,0.197177
3,0,3,-0.242048,-0.034016,-0.145236,0.179018,-0.229713,0.124971,0.101729,-0.030193,...,0.022174,0.470509,-0.195023,-0.217868,0.369650,0.219866,0.078816,0.028726,0.002379,0.157064
4,0,4,-0.084696,-0.156661,-0.001619,-0.095401,-0.225883,-0.137358,0.073911,-0.142179,...,0.078455,0.592889,-0.188250,-0.175718,0.298141,0.442099,0.283607,0.139379,-0.188109,0.234084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248851,1181,79,-0.206824,-0.085907,-0.030567,0.137892,-0.241031,0.067305,0.187766,0.045334,...,0.100099,0.576997,-0.208508,-0.294500,0.299060,0.255981,0.127127,-0.000301,0.001370,0.285149
248852,1181,80,-0.191159,0.000017,-0.094411,0.167705,-0.247148,0.095168,0.096068,0.008128,...,0.082106,0.480624,-0.189878,-0.263440,0.324670,0.212121,0.079485,0.017333,0.046133,0.195056
248853,1181,81,-0.236725,-0.047511,-0.121796,0.176346,-0.221189,0.116637,0.104347,0.086721,...,0.056311,0.569624,-0.221262,-0.255486,0.359549,0.214852,0.044585,-0.014750,0.090143,0.262733
248854,1181,82,-0.118603,-0.187553,-0.050448,-0.040236,-0.157468,0.054860,0.032803,0.034543,...,0.011561,0.410863,-0.240706,-0.197638,0.128234,0.296214,-0.027649,0.090746,-0.055693,0.275673


In [25]:
# group the rows by speech number and compute the mean of each group
df_speech = df.groupby("speech_number").mean()

# reset the index of the dataframe and rename the columns
df_speech = df_speech.reset_index().rename(columns={"index": "speech_number"})

# re-order the columns of the dataframe
df_speech = df_speech[["speech_number"] + ["dim_" + str(i) for i in range(300)]]


In [26]:
df_speech['speech_number'] = df_speech['speech_number'] +1

In [27]:
df_speech

,speech_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,1,-0.173279,-0.021146,-0.092922,0.152873,-0.268298,0.089113,0.108764,-0.014897,0.348386,...,0.119603,0.536271,-0.204575,-0.268619,0.342063,0.279612,0.123291,0.015000,0.030857,0.205801
1,2,-0.176242,-0.013237,-0.091015,0.160408,-0.269626,0.090747,0.110872,-0.023401,0.350070,...,0.124181,0.539935,-0.197811,-0.266503,0.357874,0.283431,0.124615,0.010264,0.026568,0.203215
2,3,-0.183406,-0.030252,-0.108963,0.174706,-0.284008,0.109655,0.112195,-0.001319,0.346592,...,0.121134,0.538685,-0.239016,-0.292465,0.330132,0.267172,0.104288,0.022140,0.043899,0.217252
3,4,-0.190268,-0.005764,-0.105168,0.175170,-0.263392,0.104558,0.106029,-0.008779,0.337009,...,0.110900,0.525516,-0.200887,-0.266000,0.362220,0.257156,0.109374,0.006310,0.050121,0.195509
4,5,-0.193214,-0.010700,-0.103427,0.171732,-0.261049,0.109003,0.098324,0.011304,0.331726,...,0.097081,0.522616,-0.204003,-0.275121,0.344229,0.240936,0.088049,0.011965,0.051919,0.205314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,1178,-0.188412,-0.010880,-0.110828,0.187686,-0.275341,0.118603,0.107056,-0.028657,0.350834,...,0.119488,0.522678,-0.216554,-0.276116,0.363728,0.267533,0.117928,0.015430,0.041618,0.183708
1177,1179,-0.181167,-0.029950,-0.076758,0.152011,-0.268470,0.088719,0.130105,-0.003977,0.359050,...,0.110380,0.528958,-0.205940,-0.281827,0.330856,0.266303,0.125181,0.020047,0.023837,0.206582
1178,1180,-0.208919,-0.008499,-0.119769,0.206090,-0.289219,0.132038,0.111971,0.005776,0.348469,...,0.101634,0.527860,-0.237847,-0.304361,0.358106,0.233721,0.094504,0.022492,0.061690,0.195729
1179,1181,-0.186655,-0.009658,-0.084662,0.184353,-0.284261,0.106885,0.134059,-0.010911,0.363777,...,0.122381,0.518987,-0.211778,-0.301093,0.338736,0.250693,0.126607,0.020122,0.032409,0.188715


In [28]:
df_data.head(20)

,index,Date,Speech,Title,Cleaned_Speech,Cleaned_Speech_string
0,NMS1,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",PM's Address at the Inauguration of the 104th ...,['governor andhra pradesh shri e l narasimhan ...,governor andhra pradesh shri e l narasimhan ch...
1,NMS2,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,Text of PM's address at the Inauguration of No...,['chief minister gujarat shri vijay rupani ji ...,chief minister gujarat shri vijay rupani ji co...
2,NMS3,2017-01-09,I am delighted to be here at Gift City to ina...,Text of PM’s address on the occasion of Inaugu...,['delighted gift city inaugurate india first i...,delighted gift city inaugurate india first int...
3,NMS4,2017-02-01,I congratulate the Finance Minister Arun Jaitl...,English rendering of the Prime Minister’s Stat...,['congratulate finance minister arun jaitley j...,congratulate finance minister arun jaitley jee...
4,NMS5,2017-02-05,Sri Pejavar Math’s most respected Sri Vishwesh...,English rendering of the text of PM’s address ...,['sri pejavar math respected sri vishwesh tirt...,sri pejavar math respected sri vishwesh tirth ...
5,NMS6,2017-02-21,Namaskaram. Greetings to everyone. Swami Nirvi...,Text of PM’s inaugural address (via video conf...,"['namaskaram', 'greeting everyone', 'swami nir...",namaskaram greeting everyone swami nirvinanand...
6,NMS7,2017-02-26,"My dear countrymen, Namaskar. Winter is on its...",English Translation of the text of PM’s ‘Mann ...,"['dear countryman namaskar', 'winter way', 'va...",dear countryman namaskar winter way vasant sea...
7,NMS8,2017-03-02,"Swami Chidanand Saraswati ji,Shankaracharya Di...",Text of PM’s inaugural address (via video conf...,['swami chidanand saraswati ji shankaracharya ...,swami chidanand saraswati ji shankaracharya di...
8,NMS9,2017-03-07,"Yesterday I was near mother Ganges, today I’m ...",English rendering of the text of PMs speech on...,['yesterday near mother ganges today mother na...,yesterday near mother ganges today mother narm...
9,NMS10,2017-03-07,"Chief Minister of Gujarat, Shri Vijay Rupani \...",English rendering of the text of PM's speech a...,['chief minister gujarat shri vijay rupani col...,chief minister gujarat shri vijay rupani colle...


In [29]:
df_data.drop(18, inplace=True)

In [30]:
df_data = df_data.reset_index(drop=True)

In [31]:
print(df_data.shape, df_speech.shape)

(1181, 6) (1181, 301)


In [32]:
df_speech.insert(0, "index", df_data["index"])
df_speech.insert(1, "Date", df_data["Date"])

In [33]:
df_speech

,index,Date,speech_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,NMS1,2017-01-03,1,-0.173279,-0.021146,-0.092922,0.152873,-0.268298,0.089113,0.108764,...,0.119603,0.536271,-0.204575,-0.268619,0.342063,0.279612,0.123291,0.015000,0.030857,0.205801
1,NMS2,2017-01-09,2,-0.176242,-0.013237,-0.091015,0.160408,-0.269626,0.090747,0.110872,...,0.124181,0.539935,-0.197811,-0.266503,0.357874,0.283431,0.124615,0.010264,0.026568,0.203215
2,NMS3,2017-01-09,3,-0.183406,-0.030252,-0.108963,0.174706,-0.284008,0.109655,0.112195,...,0.121134,0.538685,-0.239016,-0.292465,0.330132,0.267172,0.104288,0.022140,0.043899,0.217252
3,NMS4,2017-02-01,4,-0.190268,-0.005764,-0.105168,0.175170,-0.263392,0.104558,0.106029,...,0.110900,0.525516,-0.200887,-0.266000,0.362220,0.257156,0.109374,0.006310,0.050121,0.195509
4,NMS5,2017-02-05,5,-0.193214,-0.010700,-0.103427,0.171732,-0.261049,0.109003,0.098324,...,0.097081,0.522616,-0.204003,-0.275121,0.344229,0.240936,0.088049,0.011965,0.051919,0.205314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,NMS1178,2023-04-09,1178,-0.188412,-0.010880,-0.110828,0.187686,-0.275341,0.118603,0.107056,...,0.119488,0.522678,-0.216554,-0.276116,0.363728,0.267533,0.117928,0.015430,0.041618,0.183708
1177,NMS1179,2023-04-12,1179,-0.181167,-0.029950,-0.076758,0.152011,-0.268470,0.088719,0.130105,...,0.110380,0.528958,-0.205940,-0.281827,0.330856,0.266303,0.125181,0.020047,0.023837,0.206582
1178,NMS1180,2023-04-12,1180,-0.208919,-0.008499,-0.119769,0.206090,-0.289219,0.132038,0.111971,...,0.101634,0.527860,-0.237847,-0.304361,0.358106,0.233721,0.094504,0.022492,0.061690,0.195729
1179,NMS1181,2023-04-14,1181,-0.186655,-0.009658,-0.084662,0.184353,-0.284261,0.106885,0.134059,...,0.122381,0.518987,-0.211778,-0.301093,0.338736,0.250693,0.126607,0.020122,0.032409,0.188715


In [34]:
#generating a vector pickle
word2vec_NM = open("word2vec_NM.pickle", "wb")
pickle.dump(df_speech, word2vec_NM)
word2vec_NM.close()